<a href="https://colab.research.google.com/github/Kishan-Kumar-Zalavadia/Agentic_AI_with_Python_And_Generative_AI/blob/main/4_AIAgebtesAbdGenetiveAI_AgentDecoratrs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Keeping Agent Tools Up to Date with Python Decorators**



> Look at the AgentWithTheDecorator.ipynb file for the executed code



When building AI agents, one of the most challenging aspects is maintaining accurate documentation about the tools our agent can use. Every time we modify a tool’s parameters or behavior, we need to update its description and parameter schema. If these get out of sync, our agent might try to use tools incorrectly, leading to failures that can be hard to debug. Let’s explore how we can solve this problem elegantly using Python decorators.

#** The Challenge of Tool Documentation**

Imagine we’re writing a tool for our agent to read files. Without any special handling, we might write something like this:
```
# In our action registry setup
action_registry.register(Action(
    name="read_file",
    function=read_file,
    description="Reads content from a specified file",
    parameters={
        "type": "object",
        "properties": {
            "file_path": {"type": "string"}
        },
        "required": ["file_path"]
    }
))

# The actual function
def read_file(file_path: str) -> str:
    """Reads and returns the content of a file."""
    with open(file_path, 'r') as f:
        return f.read()
```

This approach has several problems. If we add a new parameter to read_file, we need to remember to update the parameters schema. If we change the function’s behavior, we need to update the description. It’s easy for these to become inconsistent over time.

#**The Power of Decorators**
Instead, we can use Python’s decorator system to automatically extract and maintain this information. Here’s how it works:
```
@register_tool(tags=["file_operations"])
def read_file(file_path: str) -> str:
    """Reads and returns the content of a file from the specified path.
    
    The function opens the file in read mode and returns its entire contents
    as a string. If the file doesn't exist or can't be read, it raises an
    appropriate exception.
    
    Args:
        file_path: The path to the file to read
        
    Returns:
        The contents of the file as a string
    """
    with open(file_path, 'r') as f:
        return f.read()
```
Our decorator examines the function and automatically:

1. Uses the function name as the tool name
2. Extracts the docstring for the description
3. Analyzes type hints and parameters to build the schema
4. Registers the tool in a central registry

#**Implementing the Decorator**
Let’s look at how this magic happens by first understanding the decorator and then examining the helper function that does the heavy lifting:
```
def register_tool(tool_name=None, description=None,
                 parameters_override=None, terminal=False, tags=None):
    """Registers a function as an agent tool."""
    def decorator(func):
        # Extract all metadata from the function
        metadata = get_tool_metadata(
            func=func,
            tool_name=tool_name,
            description=description,
            parameters_override=parameters_override,
            terminal=terminal,
            tags=tags
        )
        
        # Register in our global tools dictionary
        tools[metadata["tool_name"]] = {
            "description": metadata["description"],
            "parameters": metadata["parameters"],
            "function": metadata["function"],
            "terminal": metadata["terminal"],
            "tags": metadata["tags"]
        }
        
        # Also maintain a tag-based index
        for tag in metadata["tags"]:
            if tag not in tools_by_tag:
                tools_by_tag[tag] = []
            tools_by_tag[tag].append(metadata["tool_name"])
        
        return func
    return decorator
```

This register_tool decorator is designed to transform regular Python functions into callable tools for an agent system. Here’s what’s happening in detail:

1. The decorator accepts several optional parameters that configure how the function will be registered as a tool.
2. It returns an inner function (decorator) that takes the actual function being decorated (func) as its parameter.
3. Within this inner function, it first calls get_tool_metadata() to analyze and extract all relevant information from the function to turn it into a tool description that the agent can use.
4. It then registers the tool in a global dictionary called tools, using the tool name as a key. The AgentRegistry can be configured to lookup tools here.
5. For improved discovery, it also maintains a tag-based index in tools_by_tag, allowing tools to be looked up by category. This tagging allows us to define sets of tools that should be used together.
6. Finally, it returns the original function unchanged, allowing it to still be called normally while also being available as a tool.

This pattern creates a clean separation between the definition of tools and their registration, making the codebase much more maintainable.

Now, let’s examine the helper function that does the detailed analysis:
```
def get_tool_metadata(func, tool_name=None, description=None,
                     parameters_override=None, terminal=False, tags=None):
    """Extracts metadata for a function to use in tool registration."""
    
    # Use function name if no tool_name provided
    tool_name = tool_name or func.__name__
    
    # Use docstring if no description provided
    description = description or (func.__doc__.strip()
                                if func.__doc__ else "No description provided.")
    
    # If no parameter override, analyze the function
    if parameters_override is None:
        signature = inspect.signature(func)
        type_hints = get_type_hints(func)
        
        # Build JSON schema for arguments
        args_schema = {
            "type": "object",
            "properties": {},
            "required": []
        }
        
        # Examine each parameter
        for param_name, param in signature.parameters.items():
            # Skip special parameters
            if param_name in ["action_context", "action_agent"]:
                continue

            # Convert Python types to JSON schema types
            param_type = type_hints.get(param_name, str)
            param_schema = {
                "type": get_json_type(param_type)
            }
            
            args_schema["properties"][param_name] = param_schema
            
            # If parameter has no default, it's required
            if param.default == inspect.Parameter.empty:
                args_schema["required"].append(param_name)
    else:
        args_schema = parameters_override
    
    return {
        "tool_name": tool_name,
        "description": description,
        "parameters": args_schema,
        "function": func,
        "terminal": terminal,
        "tags": tags or []
    }
```

This helper function performs introspection on the decorated function to automatically extract the metadata needed to describe the tool to the agent:

1. It determines the tool’s name, defaulting to the function’s name if not explicitly provided.
2. It extracts a description from the function’s docstring if one isn’t explicitly provided.
3. For parameter handling, it does sophisticated introspection using Python’s inspect and typing modules:
  - It captures the function’s signature to identify all parameters
  - It extracts type annotations using get_type_hints()
  - It builds a JSON schema that describes the expected input structure
  - It identifies required parameters (those without default values)
  - It intelligently skips special context parameters like action_context and action_agent
  - It converts Python types to JSON schema types via a helper function get_json_type()
4. It packages all this metadata into a dictionary that the decorator can use to register the tool.

This thorough introspection enables tools to be defined with minimal boilerplate while providing rich metadata for the agent system to understand how to call each tool properly. The tool description will always match the function’s signature and docstring, ensuring that the agent has the most accurate information available.

#**Why Create a Decorator?**

You will see this decorator approach in many different agent frameworks. Here is why:

1. **Single Source of Truth**: The function itself becomes the authoritative source for all tool information. The docstring describes what it does, the type hints define its parameters, and the implementation shows how it works.

2. **Automatic Updates**: When we modify the function’s signature or documentation, the tool registration automatically stays in sync. No more hunting through code to update parameter schemas.

3. **Better Organization**: The tags system allows us to categorize tools and find related functionality. We can easily get all “file_operations” tools or all “database_tools”.

4. **Improved Development Experience**: We write our tools as normal Python functions with standard documentation. The decorator handles all the complexity of making them available to our agent.

Consider how this simplifies adding a new parameter:
```
@register_tool(tags=["file_operations"])
def read_file(file_path: str, encoding: str = 'utf-8') -> str:
    """Reads and returns the content of a file.
    
    Args:
        file_path: The path to the file to read
        encoding: The character encoding to use (default: utf-8)
    """
    with open(file_path, 'r', encoding=encoding) as f:
        return f.read()
```
The tool’s parameter schema automatically updates to include the new encoding parameter as optional (since it has a default value). We didn’t need to manually update any registration code or documentation.

Imagine you’re organizing a workshop. You wouldn’t just throw all your tools into a big box—you’d want to organize them by purpose, perhaps keeping all the measuring tools in one drawer, cutting tools in another, and so on. This is exactly what we’re doing with our agent tools using tags and registries. Let’s explore how this organization system works and how it makes our lives easier when building agents.

#**Understanding Tool Organization**

When we build agents, we often create many tools that serve different purposes. Some tools might handle file operations, others might work with databases, and still others might interact with external APIs. Our organization system has three layers:

1. Tool Decorators: Tag and document individual tools
2. Tool Registry: Central storage of all available tools
3. Action Registry: Curated sets of tools for specific agents

Let’s see how these work together.

#** Tagging Tools for Organization**
First, we use our decorator to tag tools based on their purpose:
```
@register_tool(tags=["file_operations"])
def read_file(file_path: str) -> str:
    """Reads and returns the content of a file."""
    with open(file_path, 'r') as f:
        return f.read()

@register_tool(tags=["file_operations", "write"])
def write_file(file_path: str, content: str) -> None:
    """Writes content to a file."""
    with open(file_path, 'w') as f:
        f.write(content)

@register_tool(tags=["database", "read"])
def query_database(query: str) -> List[Dict]:
    """Executes a database query and returns results."""
    return db.execute(query)
```
When we register these tools, our decorator maintains two global registries:

tools: A dictionary of all tools indexed by name
tools_by_tag: A dictionary of tool names organized by tag
```
# Internal structure of tools_by_tag
{
    "file_operations": ["read_file", "write_file"],
    "write": ["write_file"],
    "database": ["query_database"],
    "read": ["query_database"]
}
```
This organization allows us to easily find related tools. For instance, we can find all tools related to file operations or all tools that perform read operations.

#**Creating Focused Action Registries**

Now comes the powerful part. When we create an agent, we can easily build an ActionRegistry with just the tools it needs:
```
def create_file_processing_agent():
    # Create a registry with only file operation tools
    action_registry = ActionRegistry(tags=["file_operations"])
    
    return Agent(
        goals=[Goal(1, "File Processing", "Process project files")],
        agent_language=AgentFunctionCallingActionLanguage(),
        action_registry=action_registry,
        generate_response=generate_response,
        environment=Environment()
    )

def create_database_agent():
    # Create a registry with only database tools
    action_registry = ActionRegistry(tags=["database"])
    
    return Agent(
        goals=[Goal(1, "Database Operations", "Query database as needed")],
        agent_language=AgentFunctionCallingActionLanguage(),
        action_registry=action_registry,
        generate_response=generate_response,
        environment=Environment()
    )
```
#**Creating Specialized Agents**
We can create agents with very specific tool sets just by specifying tags:
```
# Create an agent that can only read (no writing)
read_only_agent = Agent(
    goals=[Goal(1, "Read Only", "Read but don't modify data")],
    agent_language=AgentFunctionCallingActionLanguage(),
    action_registry=ActionRegistry(tags=["read"]),
    generate_response=generate_response,
    environment=Environment()
)

# Create an agent that handles all file operations
file_agent = Agent(
    goals=[Goal(1, "File Handler", "Manage file operations")],
    agent_language=AgentFunctionCallingActionLanguage(),
    action_registry=ActionRegistry(tags=["file_operations"]),
    generate_response=generate_response,
    environment=Environment()
)
```

In our original README agent, we manually created and registered each action. While this approach works, it requires us to maintain the tool definitions (functions) separately from their metadata (descriptions and parameters). This separation creates opportunities for these two pieces to become out of sync. Let’s improve our agent by using tool decorators to keep everything together and automatically synchronized.

#**Understanding the Problem**
Let’s look at a piece of our original code:
```
def read_project_file(name: str) -> str:
    with open(name, "r") as f:
        return f.read()

# Later, separately, we define metadata about the function
action_registry.register(Action(
    name="read_project_file",
    function=read_project_file,
    description="Reads a file from the project.",
    parameters={
        "type": "object",
        "properties": {
            "name": {"type": "string"}
        },
        "required": ["name"]
    },
    terminal=False
))
```
In this code, if we change the function’s parameters, we need to remember to update the parameter schema. If we modify what the function does, we need to remember to update the description. These manual steps are error-prone and can lead to confusion for both developers and the LLM.

#**The Decorator Solution**
Using tool decorators, we can keep all this information together. Here’s how we’ll refactor our README agent:

```
# First, we'll define our tools using decorators
@register_tool(tags=["file_operations", "read"])
def read_project_file(name: str) -> str:
    """Reads and returns the content of a specified project file.

    Opens the file in read mode and returns its entire contents as a string.
    Raises FileNotFoundError if the file doesn't exist.

    Args:
        name: The name of the file to read

    Returns:
        The contents of the file as a string
    """
    with open(name, "r") as f:
        return f.read()

@register_tool(tags=["file_operations", "list"])
def list_project_files() -> List[str]:
    """Lists all Python files in the current project directory.

    Scans the current directory and returns a sorted list of all files
    that end with '.py'.

    Returns:
        A sorted list of Python filenames
    """
    return sorted([file for file in os.listdir(".")
                   if file.endswith(".py")])

@register_tool(tags=["system"], terminal=True)
def terminate(message: str) -> str:
    """Terminates the agent's execution with a final message.

    Args:
        message: The final message to return before terminating

    Returns:
        The message with a termination note appended
    """
    return f"{message}\nTerminating..."

def main():
    # Define the agent's goals
    goals = [
        Goal(priority=1,
             name="Gather Information",
             description="Read each file in the project in order to build a deep understanding of the project in order to write a README"),
        Goal(priority=1,
             name="Terminate",
             description="Call terminate when done and provide a complete README for the project in the message parameter")
    ]

    # Create an agent instance with tag-filtered actions
    agent = Agent(
        goals=goals,
        agent_language=AgentFunctionCallingActionLanguage(),
        # The ActionRegistry now automatically loads tools with these tags
        action_registry=PythonActionRegistry(tags=["file_operations", "system"]),
        generate_response=generate_response,
        environment=Environment()
    )

    # Run the agent with user input
    user_input = "Write a README for this project."
    final_memory = agent.run(user_input)
    print(final_memory.get_memories())

if __name__ == "__main__":
    main()
```
Let’s examine the key improvements this refactoring brings:

1. ##Self-Documenting Tools
Each tool now carries its own documentation through Python docstrings. The decorator automatically extracts this documentation and uses it as the tool’s description. This means:

- Documentation lives with the code it describes
- Changes to the function naturally lead to updates in documentation
- We get better IDE integration with docstring hints

2. ##Automatic Parameter Inference
The decorator examines the function’s type hints and signature to automatically build the parameter schema. This means:

- Parameter types stay in sync automatically
- Required parameters are detected from the function signature
- We don’t need to manually maintain a separate schema

3. ##Logical Organization Through Tags
Tools are now organized with tags that describe their purpose:

- file_operations for file-related tools
- read and list to specify the type of operation
- system for administrative functions like termination

This tagging system makes it easy to:

- Group related tools together
- Filter tools when creating agents
- Understand a tool’s purpose at a glance

4. ##Simplified Agent Creation
Notice how much simpler our agent creation becomes:

```
agent = Agent(
    goals=goals,
    agent_language=AgentFunctionCallingActionLanguage(),
    action_registry=ActionRegistry(tags=["file_operations", "system"]),
    generate_response=generate_response,
    environment=Environment()
)
```

We no longer need to manually register actions. Instead, we just specify which tags we want, and the ActionRegistry automatically includes the appropriate tools.

5. ##Easier Maintenance
When we need to modify our agent’s capabilities, we can:

- Add new tools by simply decorating functions
- Modify existing tools without touching registration code
- Change tool organization by adjusting tags
- Remove tools by removing the decorator or changing tags

# Understanding the Flow
With this refactored version:

1. When the code loads, the decorators automatically register all tools in a central registry.
2. When we create an ActionRegistry with specific tags, it automatically loads the matching tools.
3. The Agent uses these pre-configured tools without any manual registration steps.

This automation reduces errors and makes our code more maintainable. If we later want to add new capabilities to our README agent, we simply need to create new functions with the appropriate decorators and tags.